# Pulse sequence evaluation

Using arbitrary units. Energy is normalized to the standard deviation in chemical shift strengths. Reduced Planck's constant $\hbar \equiv 1$.

In [1]:
import qutip as qt
import numpy as np
import matplotlib.pyplot as plt

## Identify primitives

In [2]:
delay = 1e-2  # time is relative to chemical shift strength
pulse_width = 5e-3
N = 3  # number of spins

In [3]:
def get_Hsys(dipolar_strength=1e-2):
    chemical_shifts = 2*np.pi * np.random.normal(scale=1, size=(N,))
    Hcs = sum(
        [qt.tensor(
            [qt.identity(2)]*i
            + [chemical_shifts[i] * qt.sigmaz()]
            + [qt.identity(2)]*(N-i-1)
        ) for i in range(N)]
    )
    # dipolar interactions
    dipolar_matrix = 2*np.pi * np.random.normal(scale=dipolar_strength, size=(N, N))
    Hdip = sum([
        dipolar_matrix[i, j] * (
            2 * qt.tensor(
                [qt.identity(2)]*i
                + [qt.sigmaz()]
                + [qt.identity(2)]*(j-i-1)
                + [qt.sigmaz()]
                + [qt.identity(2)]*(N-j-1)
            )
            - qt.tensor(
                [qt.identity(2)]*i
                + [qt.sigmax()]
                + [qt.identity(2)]*(j-i-1)
                + [qt.sigmax()]
                + [qt.identity(2)]*(N-j-1)
            )
            - qt.tensor(
                [qt.identity(2)]*i
                + [qt.sigmay()]
                + [qt.identity(2)]*(j-i-1)
                + [qt.sigmay()]
                + [qt.identity(2)]*(N-j-1)
            )
        )
        for i in range(N) for j in range(i+1, N)
    ])
    return Hcs + Hdip

In [4]:
X = sum(
    [qt.tensor(
        [qt.identity(2)]*i
        + [qt.spin_Jx(1/2)]
        + [qt.identity(2)]*(N-i-1)
    ) for i in range(N)]
)
Y = sum(
    [qt.tensor(
        [qt.identity(2)]*i
        + [qt.spin_Jy(1/2)]
        + [qt.identity(2)]*(N-i-1)
    ) for i in range(N)]
)
Z = sum(
    [qt.tensor(
        [qt.identity(2)]*i
        + [qt.spin_Jz(1/2)]
        + [qt.identity(2)]*(N-i-1)
    ) for i in range(N)]
)

In [5]:
def get_pulses(Hsys, X, Y, Z, pulse_width, delay, rot_error=0):
    rot = np.random.normal(scale=rot_error)
    pulses = [
        qt.propagator(Hsys, pulse_width),
        qt.propagator(X * (np.pi/2) * (1 + rot) / pulse_width + Hsys, pulse_width),
        qt.propagator(-X * (np.pi/2) * (1 + rot) / pulse_width + Hsys, pulse_width),
        qt.propagator(Y * (np.pi/2) * (1 + rot) / pulse_width + Hsys, pulse_width),
        qt.propagator(-Y * (np.pi/2) * (1 + rot) / pulse_width + Hsys, pulse_width),
#         qt.propagator(Z * (np.pi/2) * (1 + rot) / pulse_width + Hsys, pulse_width),
#         qt.propagator(-Z * (np.pi/2) * (1 + rot) / pulse_width + Hsys, pulse_width),
    ]
    delay_propagator = qt.propagator(Hsys, delay)
    pulses = [delay_propagator * i for i in pulses]
    return pulses

In [6]:
Hsys_ensemble = [get_Hsys() for _ in range(10)]
pulses_ensemble = [
    get_pulses(H, X, Y, Z, pulse_width, delay, rot_error=0.01) for H in Hsys_ensemble
]

In [7]:
pulse_names = [
    'd', 'x', '-x', 'y', '-y', #'z', '-z'
]

In [8]:
Utarget = qt.identity(Hsys_ensemble[0].dims[0])

## Define and evaluate pulse sequences



In [9]:
def get_fidelity(pulse_sequence, Utarget, pulses):
    Uexp = qt.identity(Utarget.dims[0])
    for p in pulse_sequence:
        Uexp = pulses[p] * Uexp
    return qt.metrics.average_gate_fidelity(Uexp, Utarget)

In [10]:
def get_mean_fidelity(pulse_sequence, Utarget, pulses_ensemble):
    fidelity = 0
    for pulses in pulses_ensemble:
        fidelity += get_fidelity(pulse_sequence, Utarget, pulses)
    return fidelity / len(pulses_ensemble)

In [18]:
ideal6 = [3, 1, 1, 3, 2, 2]
yxx24 = [4, 1, 2, 3, 2, 2, 3, 2, 1, 4, 1, 1, 3, 2, 1, 4, 1, 1, 4, 1, 2, 3, 2, 2]
yxx48 = [
    3, 2, 2, 3, 2, 2, 4, 1, 1, 3, 2, 2, 4, 1, 1, 4, 1, 1, 3, 2, 2, 3, 2, 2,
    4, 1, 1, 3, 2, 2, 4, 1, 1, 4, 1, 1, 3, 2, 2, 4, 1, 1, 3, 2, 2, 4, 1, 1
]

# brute-force search
bf6 = [1, 1, 3, 1, 1, 3]
bf12 = [1, 1, 4, 1, 1, 4, 2, 2, 4, 2, 2, 4]
bfr12 = [1, 4, 4, 1, 4, 4, 1, 3, 3, 1, 3, 3]

# vanilla MCTS search
mcts12_1 = [0, 1, 0, 3, 0, 1, 0, 1, 0, 3, 0, 1]
mcts12_2 = [4, 0, 4, 1, 4, 4, 3, 4, 2, 2, 2, 0]
mcts12_3 = [4, 4, 4, 1, 0, 3, 3, 0, 1, 3, 3, 4]
mcts12_4 = [3, 0, 3, 1, 3, 0, 3, 2, 1, 1, 1, 2]
mcts24 = [4, 2, 3, 4, 2, 1, 3, 2, 0, 2, 2, 3, 4, 0, 3, 1, 2, 1, 3, 4, 1, 1, 1, 2]

In [27]:
get_mean_fidelity(mcts12_1, Utarget, pulses_ensemble)

0.9238864103809943

In [28]:
get_mean_fidelity(mcts12_2, Utarget, pulses_ensemble)

0.9804573985600967

In [29]:
get_mean_fidelity(mcts12_3, Utarget, pulses_ensemble)

0.9832478911317288

In [30]:
get_mean_fidelity(mcts12_4, Utarget, pulses_ensemble)

0.97396731685615

In [31]:
get_mean_fidelity(mcts24, Utarget, pulses_ensemble)

0.9883437990059489

In [32]:
get_mean_fidelity(ideal6, Utarget, pulses_ensemble)

0.8899080699991115

In [33]:
get_mean_fidelity(yxx24, Utarget, pulses_ensemble)

0.9979643804075797

In [34]:
get_mean_fidelity(yxx48, Utarget, pulses_ensemble)

0.9945958534354444

In [ ]:
get_mean_fidelity(((bf12 + ',')*4)[:-1], Utarget, pulses_dict_ensemble)

In [ ]:
get_mean_fidelity(((bfr12 + ',')*4)[:-1], Utarget, pulses_dict_ensemble)

In [ ]:
get_mean_fidelity(rand12, Utarget, pulses_dict_ensemble)

In [ ]:
get_mean_fidelity(','.join(['-y,x,-y,x,-y,x']*4), Utarget, pulses_dict_ensemble)

In [ ]:
fids = []
Uexp = qt.identity(Utarget.dims[0])
for p in ((bfr12 + ',')*4)[:-1].split(','):
    Uexp = pulses_dict_ensemble[1][p] * Uexp
    fids.append(qt.metrics.average_gate_fidelity(Uexp, Utarget))

In [ ]:
plt.plot(-np.log10(1-np.array(fids)))
plt.ylabel('Reward')
plt.xlabel('Pulse number')

In [ ]:
fids = []
Uexp = qt.identity(Utarget.dims[0])
for p in ((yxx48 + ',')*1)[:-1].split(','):
    Uexp = pulses_dict_ensemble[1][p] * Uexp
    fids.append(qt.metrics.average_gate_fidelity(Uexp, Utarget))

In [ ]:
plt.plot(-np.log10(1-np.array(fids)))
plt.ylabel('Reward')
plt.xlabel('Pulse number')